In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai.vision import *
from fastai.metrics import error_rate

In [0]:
bs=16

In [0]:
path=untar_data(URLs.PETS)

In [0]:
path.ls()

In [0]:
path_img=path/'images'
path_anno=path/'annotations'

In [0]:
fnames=get_image_files(path_img)
fnames[:5]

In [0]:
np.random.seed(2)
pat=r'/([^/]+)_\d+.jpg$'

In [0]:
data=ImageDataBunch.from_name_re(path_img,fnames,pat,ds_tfms=get_transforms(),size=224,bs=bs).normalize(imagenet_stats)

In [0]:
data.show_batch(rows=3,figsize=(7,6))

In [0]:
print(data.classes)
len(data.classes),data.c

In [0]:
learn=cnn_learner(data,models.resnet34,metrics=error_rate)

In [0]:
learn.model

In [0]:
learn.fit_one_cycle(4)

In [0]:
learn.save('stage-1')

In [0]:
interp=ClassificationInterpretation.from_learner(learn)
losses,idxs=interp.top_losses()
len(data.valid_ds)==len(losses)==len(idxs)

In [0]:
interp.plot_top_losses(9,figsize=(15,11))

In [0]:
interp.plot_confusion_matrix(figsize=(12,12),dpi=60)

In [0]:
interp.most_confused(min_val=2)

In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(1)

In [0]:
learn.load('stage-1')

In [0]:
learn.lr_find()

In [0]:
x=learn.recorder.plot()


In [0]:
learn.unfreeze()
learn.fit_one_cycle(2,max_lr=slice(1e-5,1e-4))
#Max Accuracy achieved uptil now: 93.9%

In [0]:
learn.load('stage-1')
learn.unfreeze()
learn.fit_one_cycle(2,max_lr=slice(1e-4,0.2e-4))
#Accuracy:93.5%

Now,using Resnet50


In [0]:
data=ImageDataBunch.from_name_re(path_img,fnames,pat,ds_tfms=get_transforms(),size=299,bs=16).normalize(imagenet_stats)

In [0]:
learn=cnn_learner(data,models.resnet50,metrics=error_rate)

In [0]:
learn.model

In [0]:
learn.lr_find()
learn.recorder.plot()


In [0]:
learn.fit_one_cycle(4)

In [0]:
learn.save('1-r50')

In [0]:
learn.unfreeze()
learn.fit_one_cycle(4,max_lr=slice(1e-6,1e-4))
#New accuracy=95.4%